In [1055]:
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import gymnasium as gym
from gymnasium import spaces


from typing import Any, Literal
from pandas import DataFrame


import torch
Tensor = torch.Tensor
import torch.nn as nn
from torch.optim import Adam, RMSprop


In [1056]:
path_m5 : str = r"C:\Users\Nemo\Desktop\sth-financial\XAUUSDM5.csv"
path_m15: str = r"C:\Users\Nemo\Desktop\sth-financial\XAUUSDM15.csv"

In [1057]:
df = pd.read_csv(path_m15 , 
				   encoding = 'utf-16', 
				   names =  ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', '0'] )

df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', drop=True, inplace = True)
df.drop('0', axis=1)


# variables

Open_ = df['Open']
close = df['Close']
# volume = data['Volume']
high = df['High']
time = df.index

Round = 4

rsi_len = 14

std_len = mean_len = 5

Vol_ma_1 = ma_1 = 10

Vol_ma_2 = ma_2 = 20


# X = df[[ 'change', 'rsi',   'macd' ]]
# Y = df[[ 'buy', 'sell']]

# X_train, X_test, y_train, y_test = \
# train_test_split( X,
#                  Y, 
#                  test_size= 0.30
#                  , shuffle = True)

In [1058]:
df.head()

,Open,High,Low,Close,Volume,0
Date,,,,,,
2025-04-16 01:00:00,3230.96,3233.35,3229.84,3232.16,482,0
2025-04-16 01:15:00,3232.15,3235.25,3231.47,3235.25,368,0
2025-04-16 01:30:00,3234.82,3238.90,3234.37,3238.82,733,0
2025-04-16 01:45:00,3238.76,3242.23,3237.48,3241.47,752,0
2025-04-16 02:00:00,3241.52,3241.74,3238.36,3241.73,756,0


#3

In [1059]:


from typing import Any
from numpy import dtype, floating
from pandas.core.frame import DataFrame
from pandas.core.series import Series




def calculate_RSI(data: Series , time_window : int = 14) -> Series:
		delta: Series[type[float]] = data.diff() # type: ignore

		up, down = delta.copy(), delta.copy()
		up[up < 0] = 0 # type: ignore
		down[down > 0] = 0 # type: ignore

		roll_up: Series[float] = up.rolling(time_window).mean()
		roll_down: Series[float] = down.abs().rolling(time_window).mean()

		RS: Series[float] = roll_up / roll_down
		RSI: Series[float] = 100.0 - (100.0 / (1.0 + RS))

		return RSI.round(Round)

def norming(row : pd.Series, on: str ) -> Literal[1] | Literal[0]:
	if max(row['buy'], row['sell'], row['hold']) == row[on]:
		return 1
	else:
		return 0

# def ABS(row : DataFrame ) -> pd.Series:
# 	if row['sell']:
# 		return row["sell"].abs()
# 	else:
		# return row['sell']

def moving_avg(data : np.ndarray | pd.Series , window_size : int)  :
	if isinstance(data, np.ndarray):
		return np.convolve(data, np.ones(window_size)/window_size, mode='valid').round(Round)
	if isinstance(data, pd.Series):
		return data.rolling(window=window_size, min_periods=1).mean().round(Round)

# def macd(data, fast, slow):
#     if isinstance(data, np.ndarray):
#         ma1 =  np.convolve(data, np.ones(fast)/fast, mode='valid')
#         ma2 =  np.convolve(data, np.ones(slow)/slow, mode='valid')
#         return ma2 - ma1
#     if isinstance(data, pd.Series):
#         ma1 =  data.rolling(fast, min_periods=1).mean()
#         ma2 =  data.rolling(slow, min_periods=1).mean()
#         return ma2 - ma1

# def clc_std(series : pd.Series , window: int):
#     rolling_std = series.rolling(window=window).std()
#     return  rolling_std

def clc_mean(series : Series , window: int) :
		rolling_mean = series.rolling(window=window).mean().round(Round)
		return  rolling_mean

def moving_avg_std(series : Series , window : int)  -> Series:
	"""Calculate the moving average and standard deviation of a pandas series, and then divide the two"""
	rolling_mean = series.rolling(window=window).mean().round(Round)
	rolling_std = series.rolling(window=window).std().round(Round)
	return rolling_mean - rolling_std
				
# def lr_schedule_1(epoch, lr):
#     if epoch < 3:
#         return lr
#     else:
#         return lr * tf.math.exp(-0.1)

# def lr_schedule_2(epoch, lr, decay_rate=0.5, decay_steps = 3 ):
#     return lr * decay_rate**(np.floor(epoch / decay_steps))

In [1060]:
def creat_data(df: pd.DataFrame):
	df['diff']  = round( df['High'] - df['Open'] , Round )
	df['cadle-type']  = (df['Close'] - df['Open']).round(Round)
	df['hike'] = df['diff'] / df['cadle-type']
	df['change']  =  df['Open'] - moving_avg(df['Open'], ma_1 ) 

	df['percent']  =  (close - df['Open']) / close  * 100

	df['rsi']  = calculate_RSI( df['Open'] , rsi_len ) / 100

	# df['std'] = Std = round(std(df['Open'] , std_len), Round )

	df['mean'] = clc_mean(df['Open'] , std_len)

	df['buy'] = np.where( (df['Open'] > clc_mean(df['Open'],mean_len).shift(1)) & (df['cadle-type'] >= .65), 1 ,0 )

	df['sell'] = np.where( (df['Open'] < clc_mean(df['Open'],mean_len).shift(1)) & (abs(df['cadle-type']) >= .65)  , 1 ,0 )

	df['hold'] = np.where(df['buy'] == df['sell'], 1, 0)

	# df['action'] = np.where(buyers == 1, 1,0)

	df = df.dropna(axis=0)

	return df

In [1061]:
df = creat_data(df)

# df.tail()

data = df[
					['Open', 'High', 'Low' , 'Close', 'Volume', 'diff' ]
					]
data.head()

,Open,High,Low,Close,Volume,diff
Date,,,,,,
2025-04-16 04:30:00,3264.93,3266.70,3256.08,3265.71,1258,1.77
2025-04-16 04:45:00,3265.70,3271.47,3265.41,3270.23,1053,5.77
2025-04-16 05:00:00,3270.64,3275.35,3269.00,3271.31,1187,4.71
2025-04-16 05:15:00,3271.37,3273.08,3269.88,3271.71,940,1.71
2025-04-16 05:30:00,3271.65,3274.14,3268.86,3269.35,1002,2.49


In [1062]:

class MarketEnv(gym.Env):
	metadata = {"render_modes": ["human"], "render_fps": 4}

	def __init__(self, df: pd.DataFrame, win_size: int):
		super().__init__()
		self.df = df.reset_index(drop=True).astype(np.float32)
		self.win_size = win_size
		self.num_features = df.shape[1]
		self.current_step = self.win_size
		self.max_steps = len(df) - win_size - 1
		self.step_count : int = 0
		self.current_position : int = 0
		self.reward : float = 0.
		self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)
		self.observation_space = spaces.Box(
			low=-np.inf,
			high=np.inf,
			shape=(win_size, self.num_features),
			dtype=np.float32
		)
		self.state : np.ndarray

	def reset(self, *, seed=None, options=None):
		super().reset(seed=seed)
		self.current_step = self.win_size
		self.step_count = 0
		self.current_position = 0
		self.state = self._get_observation()
		# print(f"state\n{self.state}")
		return self.state, {}

	def _get_observation(self):
		window = self.df.iloc[self.current_step - self.win_size : self.current_step]
		return window.to_numpy()

	def step(self, action : int, margin : float):

		# action += 1
		match action:
			case  1:
				action = 1
			case 2:
				action = -1
			case 3:
				action = 0
		# State:
		# O: 0 | H: 1 | L: 2 | C: 3 | V: 4 | D: 5 |
		self.reward = self.state[:,5].sum() * (action )
		# print(self.reward, action )
		# print(f"reward\n{self.reward}")
		self.current_step += 1
		self.step_count += 1
		self.current_position = action # type: ignore
		terminated = False
		truncated = self.step_count >= self.max_steps
		self.state = self._get_observation()

		info = {}
		return self.state, self.reward , terminated, truncated, info

	def render(self, mode="human"):
		print(f"Step: {self.step_count}, Reward: {self.reward:.4f}")


In [1063]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class Actor(nn.Module):
	def __init__(self, win_size , n_feature):
		super().__init__()

		self.norm_all = nn.LayerNorm(n_feature)
		
		self.i_know = nn.LSTM(input_size=n_feature, 
						hidden_size=n_feature,
						  num_layers= 128,
						  batch_first=True)
		
		self.i_see = nn.Sequential(
			nn.Conv1d(in_channels=win_size, out_channels=win_size, kernel_size=3, padding='same'),
			nn.BatchNorm1d(win_size),
			nn.ReLU()
			)

		self.i_decide = nn.Sequential(
			nn.Flatten(),
			# nn.LayerNorm(input_dim),
			nn.Linear(win_size * n_feature, 128), # fc1
			nn.LeakyReLU(),
			nn.Linear(128, 128), # fc2
			nn.LeakyReLU(),
			nn.Linear(128, 3), # out
			nn.Softmax()         
		)


	def forward(self, X: Tensor) -> Tensor:

		x : Tensor = self.norm_all(X).unsqueeze(0)
		# print(x.shape)

		iknow,_ = self.i_know(x)

		isee : Tensor = self.i_see(x)
		# print(isee.shape, iknow.shape)
		whathpend = iknow.tanh() + isee
		# print(whathpend.shape)
		# print(whathpend.flatten().shape)

		out = self.i_decide(whathpend)

		return out


class Critic(nn.Module):
	def __init__(self, state_dim: int, action_dim: int):
		super().__init__()
		input_dim = state_dim + action_dim

		self.fc = nn.Sequential(
			nn.LayerNorm(input_dim),
			nn.Linear(input_dim, 256),
			nn.InstanceNorm1d(256),
			nn.LeakyReLU(),
			nn.Linear(256, 128),
			nn.InstanceNorm1d(128),
			nn.LeakyReLU(),
			nn.Linear(128, 1)  # Q-value output
		)

	def forward(self, state: Tensor, action: Tensor) -> Tensor:
		# action = action.unsqueeze(0) if action.ndim == 1 else action
		# print(state.shape, action.shape)

		# if state.ndim == 1:
		# 	# batch mode
		# 	x = torch.cat([state, action], dim=1)
		# else:
		# 	# single sample mode
		# 	x = torch.cat([state, action], dim=0)
		x = torch.cat([state.flatten(), action.unsqueeze(0)]).unsqueeze(0)
		value: Tensor = self.fc(x)
		return value


# تبدیل observation به بردار 1D برای ورودی شبکه
def preprocess_obs(obs : np.ndarray | Tensor) -> torch.Tensor:
	return torch.tensor(obs, dtype=torch.float32)


In [1064]:
assert not data.isnull().any().any(), "NaN exists in dataframe!"

data.describe()

,Open,High,Low,Close,Volume,diff
count,5395.000000,5395.000000,5395.000000,5395.000000,5395.000000,5395.000000
mean,3322.211231,3325.043720,3319.199804,3322.209398,983.464690,2.832489
std,55.558108,55.443088,55.595445,55.556601,248.673301,2.831925
min,3126.920000,3132.400000,3120.780000,3126.940000,106.000000,0.000000
25%,3294.825000,3297.890000,3292.210000,3294.845000,839.000000,0.910000
50%,3326.500000,3329.100000,3323.780000,3326.510000,991.000000,2.050000
75%,3357.625000,3360.565000,3355.080000,3357.705000,1146.000000,3.820000
max,3497.360000,3499.980000,3489.180000,3497.330000,1966.000000,35.140000


In [1065]:
torch.autograd.set_detect_anomaly(True)


In [1066]:

window_size = 5

env = MarketEnv(df=data, win_size=window_size)
obs, _ = env.reset()

actor: Actor   = Actor (win_size=window_size, n_feature=env.num_features )
critic: Critic = Critic(state_dim=window_size * env.num_features, action_dim=1)

optimizerA = optim.Adam(actor.parameters(), lr=1e-3)
optimizerC = optim.Adam(critic.parameters(), lr=1e-3)

gamma = 0.99
num_episodes = 10
max_steps = 1_00
reward_to_episodes = []

for episode in range(num_episodes):
	actor.train()
	critic.train()
	
	obs, _ = env.reset()
	done = False
	total_reward = 0
	
	for step in range(max_steps):
		state: Tensor = preprocess_obs(obs)
		# print(f"state\n{state}")
		# Choosing Act
		action_probs: Tensor = actor(state)
		print(f"probs {action_probs}")
		# dist =  torch.distributions.Categorical(action_probs)
		# print(f"dist {dist.logits}")
		action: Tensor = torch.argmax(action_probs) +1 #dist.sample()
		print(f"action {action}")
		margin: Tensor = action_probs.max().round(decimals=2)
		# print(f"margin {margin}")
		
		# اجرای عمل در محیط
		next_obs, reward, terminated, truncated, info = env.step(action.item(), margin.item())
		done = terminated or truncated
		
	   
		next_state = preprocess_obs(next_obs)
		
		# محاسبه ارزش حالت فعلی و بعدی
		# print(state.shape)
		q_val   : Tensor = critic(state,action).detach().requires_grad_()
		# print(f"value {value}")
		next_q_value: Tensor = critic(next_state,action).detach().requires_grad_()
		
		# target critic Q-value
		with torch.no_grad():
			next_action = torch.argmax(actor(next_state)+1)
			# print(next_action, )
			target_q = reward + gamma * critic(next_state, next_action) * (1.0 - float(done))
		
		
		# critic loss
		predicted_q = critic(state, action)
		critic_loss = F.mse_loss(predicted_q, target_q)

		
		# actor loss
		# predicted_action = actor(state)
		actor_loss: Tensor = -critic(state, action).mean().detach().requires_grad_()
		
		# بهروزرسانی Critic
		optimizerC.zero_grad()
		critic_loss.backward(retain_graph=True)
		optimizerC.step()
		
		# بهروزرسانی Actor
		optimizerA.zero_grad()
		actor_loss.backward(retain_graph=False)
		optimizerA.step()
		
		total_reward += reward.item()
		obs = next_obs
		# print("check")
		env.render()
		if done:
			break
	reward_to_episodes.append(total_reward)
	print(f"Episode {episode} | Total Reward: {total_reward}")

probs tensor([[0.3325, 0.3402, 0.3274]], grad_fn=<SoftmaxBackward0>)
action 2
Step: 1, Reward: -16.4500
probs tensor([[0.3326, 0.3412, 0.3263]], grad_fn=<SoftmaxBackward0>)
action 2
Step: 2, Reward: -15.7100
probs tensor([[0.3323, 0.3404, 0.3274]], grad_fn=<SoftmaxBackward0>)
action 2
Step: 3, Reward: -15.5700
probs tensor([[0.3319, 0.3410, 0.3272]], grad_fn=<SoftmaxBackward0>)
action 2
Step: 4, Reward: -15.2100
probs tensor([[0.3321, 0.3408, 0.3272]], grad_fn=<SoftmaxBackward0>)
action 2
Step: 5, Reward: -14.4800
probs tensor([[0.3320, 0.3409, 0.3270]], grad_fn=<SoftmaxBackward0>)
action 2
Step: 6, Reward: -17.4900
probs tensor([[0.3321, 0.3407, 0.3272]], grad_fn=<SoftmaxBackward0>)
action 2
Step: 7, Reward: -18.6900
probs tensor([[0.3316, 0.3405, 0.3279]], grad_fn=<SoftmaxBackward0>)
action 2
Step: 8, Reward: -13.0600
probs tensor([[0.3315, 0.3414, 0.3272]], grad_fn=<SoftmaxBackward0>)
action 2
Step: 9, Reward: -17.1400
probs tensor([[0.3315, 0.3407, 0.3278]], grad_fn=<SoftmaxBackwar

KeyboardInterrupt: 

In [ ]:
import seaborn as sns

sns.lineplot({"reward": reward_to_episodes})
plt.title("total reward for each episode")
plt.xlabel('Episodes')
plt.ylabel("Rewards")
plt.grid()
plt.show()

In [ ]:


data_tensor = torch.tensor(data.to_numpy(), dtype=torch.float).reshape(-1, 5,6)
# print(data_tensor)

labels = []
with torch.no_grad():
	actor.eval()
	for state in data_tensor:
		Output = actor(state.flatten())
		Output = torch.argmax(Output)
		labels.append(Output)

labels = torch.tensor(labels)
print(labels)
print(Output)

In [ ]:
torch.unique(labels), labels.shape , data_tensor.shape, data.shape 